In [1]:
%matplotlib widget
from IPython import get_ipython
ipython = get_ipython()
if '__IPYTHON__' in globals():
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')

import argparse
import os
from glob import glob
import pickle
from itertools import compress

from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
from scipy.io import loadmat

import tensorflow as tf


from mymodels.macromicro_conv_model import load_matlab, shifted_zscore, print_metric, zscore

def minmax(x, inverse=False):
    xmin = np.array([-1,-1,6.8,-1])
    xmax = np.array([4000.,4000.,8.,4000.])
    
    a = xmax - xmin

    if inverse:
        x = x*a + xmin
    else:
        x = (x-xmin)/a

    return x


# normalize_data = minmax
# revert_data = lambda x: minmax(x, inverse=True)

# normalize_data = shifted_zscore_cf
# revert_data = lambda x: shifted_zscore_cf(x, inverse=True)

normalize_data = lambda x: x
revert_data = lambda x: x

# this is actually the number of records per UNIQUE CONCENTRATIONS per probe
n_records_per_probe = -1 # all
# n_records_per_probe = 1



Instructions for updating:
non-resource variables are not supported in the long term


In [30]:

names = ['DA', '5HT', 'pH', 'NE']
speed = 'slow'
prefix = '/mnt/nfs/proj/in-vitro/Leonardo/cf_data'

good_probes = [
    'CF025', 'CF027', 'CF057', 'CF064', 'CF066', 'CF078', 'CF081', 'CF082'
]

# jitter = None
jitter = 50
probe_data = {}
last_conc = None
for (iprobe, probe) in enumerate(good_probes):

    print('loading probe %s'%probe)
    probe_file = os.path.join(prefix, f'{probe}.npz')
    np_probe = np.load(probe_file)
    x_probe = np_probe['x']
    y_probe = np_probe['y']

    # get n unique records per label
    if n_records_per_probe > -1:
        _, ulidx = np.unique(y_probe, return_index=True, axis=0)
        u_x = []
        u_y = []
        for idx in ulidx:
            all_idx = np.where((y_probe == y_probe[idx,:]).all(axis=1))[0]
            u_x.append(x_probe[all_idx[:n_records_per_probe], :])
            u_y.append(y_probe[all_idx[:n_records_per_probe], :])
        x_probe = np.concatenate(u_x)
        y_probe = np.concatenate(u_y)

    y_probe = np.apply_along_axis(normalize_data, axis=1, arr=y_probe) 

    ul, ulidx = np.unique(y_probe, return_index=True, axis=0)
    if last_conc is not None:
        if np.any(ul.flatten() - last_conc.flatten()):
            print('%s different'%probe)
        last_conb = ul
    else:
        last_conc = ul
    
    if jitter is not None:
        for (ix, x) in enumerate(x_probe):
            j = np.random.randint(-jitter, jitter)
            y = np.roll(x, j)
            if j > 0:
                y[:j] = 0
            elif j < 0:
                y[j:] = 0
            x_probe[ix,:] = y

    probe_data[probe] = (x_probe, y_probe)


loading probe CF025
loading probe CF027
CF027 different
loading probe CF057
CF057 different
loading probe CF064
CF064 different
loading probe CF066
CF066 different
loading probe CF078
CF078 different
loading probe CF081
CF081 different
loading probe CF082


In [31]:
import time

fig, axs = plt.subplots(tight_layout=True, figsize=(12, 5))

# fig,ax = plt.subplots(1,1)
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_xlim(0,1)
# ax.set_ylim(0,1)

# def pltsin(ax, colors=['b']):
#     x = np.linspace(0,1,100)
#     if ax.lines:
#         for line in ax.lines:
#             line.set_xdata(x)
#             y = np.random.random(size=(100,1))
#             line.set_ydata(y)
#     else:
#         for color in colors:
#             y = np.random.random(size=(100,1))
#             ax.plot(x, y, color)
#     fig.canvas.draw()
    
# fig,ax = plt.subplots(1,1)
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_xlim(0,1)
# ax.set_ylim(0,1)
# for f in range(5):
#     pltsin(ax, ['b', 'r'])
#     time.sleep(1)
    

plot_data = np.zeros((list(probe_data.values())[0][0].shape[1], len(good_probes)))
for (iprobe, probe) in enumerate(good_probes):
    probe_x, probe_y = probe_data[probe]
    ul, ulidx = np.unique(probe_y, return_index=True, axis=0)
    plot_data[:,iprobe] = np.mean(probe_x[ulidx,:], axis=0)

axs.plot(plot_data)
# axs.set_ylim(-50, 50)

# for shift in range(0,900,5):
#     axs.plot(plot_data[:,shift:shift+40], label=probe)
#     plt.draw()
#     time.sleep(.5)
axs.legend(good_probes)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
for (iprobe, probe) in enumerate(good_probes):
    probe_x, probe_y = probe_data[probe]

    ul, ulidx = np.unique(probe_y, return_index=True, axis=0)
    
    fig, axs = plt.subplots(tight_layout=True, figsize=(12, 5))

    for idx in ulidx:
        axs.plot(probe_x[idx,:])
    axs.set_title(probe)
#     axs[iprobe].set_ylim(-50, 50)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
fig, axs = plt.subplots(2, 2, sharey=False, sharex=False, tight_layout=True)
fig.set_size_inches(10,10)

last_probe_y = None
for probe in good_probes:
    _ , probe_y = probe_data[probe]

#     print(probe, np.mean(probe_y, axis=0))
#     print(probe, np.std(probe_y, axis=0))
    
    std = np.std(probe_y, axis=0)
    print(probe, np.mean(probe_y, axis=0)-10*std)
    print(probe, std)

    print()

#     zeros = normalize_data(np.zeros(4))
    for ic in range(0,4):
        idx = np.unravel_index(ic, axs.shape)
        x = probe_y[:,ic]
        axs[idx].hist(x, alpha=.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

CF025 [-1.2690032e+04 -1.2688903e+04  6.0043130e+00 -1.2694118e+04]
CF025 [1.3577971e+03 1.3577040e+03 1.3915135e-01 1.3582355e+03]

CF027 [-1.26871455e+04 -1.26977432e+04  6.00474358e+00 -1.26833506e+04]
CF027 [1.3574913e+03 1.3585759e+03 1.3913688e-01 1.3571090e+03]

CF057 [-1.2688219e+04 -1.2693567e+04  6.0042143e+00 -1.2690090e+04]
CF057 [1.3576005e+03 1.3581786e+03 1.3914376e-01 1.3577697e+03]

CF064 [-1.2685267e+04 -1.2694261e+04  6.0044909e+00 -1.2691211e+04]
CF064 [1.3573193e+03 1.3582217e+03 1.3914590e-01 1.3579270e+03]

CF066 [-1.2685498e+04 -1.2694119e+04  6.0043459e+00 -1.2691737e+04]
CF066 [1.3573386e+03 1.3582080e+03 1.3915472e-01 1.3579791e+03]

CF078 [-1.2685906e+04 -1.2687273e+04  6.0045390e+00 -1.2692064e+04]
CF078 [1.3573467e+03 1.3574780e+03 1.3914447e-01 1.3580123e+03]

CF081 [-1.2686164e+04 -1.2687549e+04  6.0044622e+00 -1.2692280e+04]
CF081 [1.3573716e+03 1.3575029e+03 1.3915764e-01 1.3580276e+03]

CF082 [-1.2690922e+04 -1.2688216e+04  6.0043612e+00 -1.2693766e+0

In [6]:
np.sqrt(288992.4375)

537.5801684400197

In [7]:
(693.100830078125 + 639.6287231445312 + 7.405788421630859 + 516.1419677734375)/4

464.06932735443115